In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import numpy as np
from puzzle import Point, Puzzle

In [10]:
puz = Puzzle("inputs.txt")
puz.part_1()

504

In [11]:
puz.part_2()

1558722

In [15]:
for row in puz.basins:
    print("".join(" " if x == 0 else "*" for x in row))

****** ***  *  ****** ******** *** *** ********  ****** ***** ** **** ***** ******* *********** *** 
***** ***  *** ******* ****** * * * * ******** ** ***  *******  ******  ** * ***** ************* ***
**** **** ***** ***** **** * *** ***   *********** * * ******   ********  *******  ***********  ****
**  **********   ***  *** * ********* ***********  * ********* ******* **  ***** ***********  ******
* * *********** *** ** * **  *******   ********* **  ***** **** **********   **** * ********* ***** 
 *** *********** * **** ***** ******* *********** *** **  * ** ********** * ****** ***********  ** *
**** **********   **** * **** *****  *  * ****** *****  ****    ******** *** *****  ******* * **  **
 ** **********  * **** *******  *** ** * * ***  * ***  ****** ********* ***** * * *  * *** * * * *  
* *  **  ***** *** ** ******* ** ***  *****  * *** * * *****   ********* *****  * *** *** ****  ****
** **   ******* ***  ************ *  *******  ***   *** *****    ****** ****  ** ***** * **

In [22]:
fname = "tests.txt"
raw = Path(fname).open().readlines()
grid = np.array([list(row.strip()) for row in raw]).astype(int)


low_pts = []

for rownum, row in enumerate(grid):
    for colnum, val in enumerate(row):
        pt = Point(rownum, colnum, grid)
        if pt.is_lowest():
            low_pts.append(pt)
pt

Point(row=4, col=9)

In [57]:
basins = np.where(grid == 9, 0, 1)
basins

array([[1, 1, 0, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 0, 1, 0, 1, 1],
       [0, 1, 1, 1, 1, 1, 0, 1, 0, 1],
       [1, 1, 1, 1, 1, 0, 1, 1, 1, 0],
       [0, 1, 0, 0, 0, 1, 1, 1, 1, 1]])

In [60]:
from scipy.ndimage import measurements

lw, num = measurements.label(basins)
area = measurements.sum(basins, lw, index=np.arange(lw.max() + 1))
area

array([ 0.,  3.,  9., 14.,  9.])

## Black format my final answer

In [67]:
from pathlib import Path
from dataclasses import dataclass, field
import numpy as np
from scipy.ndimage import measurements


T1_ANS = 15
T2_ANS = 1134


@dataclass
class Point:
    row: int
    col: int
    grid: np.array = field(repr=False)

    @property
    def value(self):
        return self.grid[self.row, self.col]

    @property
    def lt_up(self):
        if self.row != 0:
            return self.value < self.grid[self.row - 1, self.col]
        return True

    @property
    def lt_down(self):
        if self.row != (len(self.grid) - 1):
            return self.value < self.grid[self.row + 1, self.col]
        return True

    @property
    def lt_left(self):
        if self.col != 0:
            return self.value < self.grid[self.row, self.col - 1]
        return True

    @property
    def lt_right(self):
        if self.col != (len(self.grid[0]) - 1):
            return self.value < self.grid[self.row, self.col + 1]
        return True

    def is_lowest(self):
        return all([self.lt_up, self.lt_down, self.lt_left, self.lt_right])


@dataclass
class Puzzle:
    fname: str
    grid: np.array = None

    def __post_init__(self):
        raw = Path(self.fname).open().readlines()
        self.grid = np.array([list(row.strip()) for row in raw]).astype(int)

    def find_low_pts(self):
        low_pts = []

        for rownum, row in enumerate(self.grid):
            for colnum, val in enumerate(row):
                pt = Point(rownum, colnum, self.grid)
                if pt.is_lowest():
                    low_pts.append(pt)

        return low_pts

    def part_1(self):
        low_pts = self.find_low_pts()
        return sum([pt.value + 1 for pt in low_pts])

    @property
    def basins(self):
        return np.where(self.grid == 9, 0, 1)

    def basin_areas(self):
        lw, num = measurements.label(self.basins)
        areas = measurements.sum(self.basins, lw, index=np.arange(lw.max() + 1))
        return areas.astype(int)

    def part_2(self):
        areas = self.basin_areas()
        top_3 = sorted(areas)[-3:]
        return np.product(top_3)


def run_tests(p1_ans=T1_ANS, p2_ans=T2_ANS, fname="tests.txt"):
    puz = Puzzle(fname)
    t1 = puz.part_1()
    assert t1 == p1_ans, f"Test 1 failed. Got {t1} instead of {p1_ans}"

    if p2_ans is not None:
        t2 = puz.part_2()
        assert t2 == p2_ans, f"Test 2 failed. Got {t2} instead of {p2_ans}"

    print("All tests passed.")
    return


if __name__ == "__main__":
    run_tests()

    puz = Puzzle("inputs.txt")

    p1 = puz.part_1()
    print("Part 1:", p1)

    if T2_ANS is not None:
        p2 = puz.part_2()
        print("Part 2:", p2)

All tests passed.
Part 1: 504
Part 2: 1558722


In [5]:
import numpy as np
from scipy import ndimage

# floor = np.array(
#     [
#         [2, 1, 9, 9, 9, 4, 3, 2, 1, 0],
#         [3, 9, 8, 7, 8, 9, 4, 9, 2, 1],
#         [9, 8, 5, 6, 7, 8, 9, 8, 9, 2],
#         [8, 7, 6, 7, 8, 9, 6, 7, 8, 9],
#         [9, 8, 9, 9, 9, 6, 5, 6, 7, 8],
#     ]
# )

floor = puz.grid

mask = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]])

window_minima = ndimage.minimum_filter(floor, footprint=mask, mode="constant", cval=9)
minima = floor[floor == window_minima]
sum(minima + 1)

1654

In [6]:
np.where(floor == window_minima, 1, 0)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])